# How to get UI dataset


In [1]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
import torch
import joblib
import os

#tf denpendencies
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Activation,Input,Bidirectional
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.utils import plot_model,to_categorical

#plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#google drive
from google.colab import drive
drive.mount('/content/gdrive')

!pip install scikit-tensor-py3
from sktensor import dtensor
# !pip install pyngrok
# from pyngrok import ngrok

Mounted at /content/gdrive
     |████████████████████████████████| 51kB 3.4MB/s 
     |████████████████████████████████| 25.2MB 2.0MB/s 
     |████████████████████████████████| 17.4MB 254kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.6 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.3.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [3]:
# Load raw train data
file_path_test = '/content/gdrive/My Drive/COMP5703/Dataset/[1109]TrainData_adjusted.csv'
# https://drive.google.com/file/d/1gJTFdyq_yfRK1-PD3Y_S-UkDJIrseBVu/view?usp=sharing
cols = ['id', 'userId', 'gender', 'sport',
          'calories', 'Route_id', 'distance_adjusted_sum','altitude_adjusted','distance_adjusted','speed_adjusted','heart_rate_adjusted','derived_speed','distance_adjusted_sum','longitude','latitude']
df = pd.read_csv(file_path_test, usecols=cols)
df.rename(columns={'speed_adjusted': 'speed','altitude_adjusted':'altitude','distance_adjusted':'derived_distance','heart_rate_adjusted':'heart_rate'},inplace=True)

In [4]:
# Load raw test data
file1_path_test = '/content/gdrive/My Drive/COMP5703/Dataset/[1109]TestData_adjusted.csv'
# https://drive.google.com/file/d/1-4WQdpOogRog7Jg_sP5SrOb802UTqpGc/view?usp=sharing
df1 = pd.read_csv(file1_path_test, usecols=cols)
df1.rename(columns={'speed_adjusted': 'speed','altitude_adjusted':'altitude','distance_adjusted':'derived_distance','heart_rate_adjusted':'heart_rate'},inplace=True)

In [5]:
# Combination
df = pd.concat( [df, df1], axis=0 )
print(len(df))

62831


In [6]:
# Define a fuction to find top 2 sport id to insert username
def get_top2id(df, gender, sporttype):
  df_new = df[(df['gender'] == gender) & (df['sport']==sporttype)]
  d1 = df_new.userId.value_counts()
  x1 = d1.index[:2].tolist()
  return df_new, x1

In [7]:
d1, male_runid = get_top2id(df, 'male', 'run')
d2, male_bikeid = get_top2id(df, 'male', 'bike')
d3, male_mountainbikeid = get_top2id(df, 'male', 'mountain bike')
d4, female_runid = get_top2id(df, 'female', 'run')
d5, female_bikeid = get_top2id(df, 'female', 'bike')
d6, female_mountain_bikeid = get_top2id(df, 'female', 'mountain bike')

In [8]:
# We found top 2 id in female mountain bike has been overlapped, so manually adjusted
d6 = df[df['gender'] == 'female']
d6 = d6[d6['sport'] == 'mountain bike']
d6_1 = d6.userId.value_counts()
x6_1 = d6_1.index[1:2].tolist()
x6_2 = d6_1.index[3:4].tolist()
female_mountain_bikeid = x6_1+x6_2
print(female_mountain_bikeid)

[6892365, 4203888]


In [9]:
# Combination of dictionary
sport_id_dict = male_runid+male_bikeid+male_mountainbikeid+female_runid+female_bikeid+female_mountain_bikeid

In [10]:
sport_id_dict

[1063624,
 860533,
 3714939,
 9051351,
 171814,
 58536,
 368603,
 7178673,
 240120,
 4164701,
 6892365,
 4203888]

In [11]:
# Give 12 different user names by gender
names= ['Bruce','Christian','Dennis','Elton','Franklin','George','Catherine','Emma','Judy','Milly','Serena','Wendy']

In [12]:
# Get user_id - name
dictionary = dict(zip(sport_id_dict, names))
print(dictionary)

{1063624: 'Bruce', 860533: 'Christian', 3714939: 'Dennis', 9051351: 'Elton', 171814: 'Franklin', 58536: 'George', 368603: 'Catherine', 7178673: 'Emma', 240120: 'Judy', 4164701: 'Milly', 6892365: 'Serena', 4203888: 'Wendy'}


In [13]:
df = df.loc[df['userId'].isin(sport_id_dict)]
df['name'] = df['userId'].map(dictionary)
print(len(df))

4500


In [14]:
df

,id,userId,gender,sport,calories,longitude,latitude,derived_speed,Route_id,altitude,derived_distance,speed,heart_rate,distance_adjusted_sum,name
8,236905945,9051351,male,bike,1762.83,"[-96.07764498330653, -96.07759008184075, -96.0...","[41.23508454300463, 41.236099088564515, 41.236...","[29.044920055943216, 37.23132633001016, 37.609...",1,"[442.2, 440.6, 437.2, 433.6, 431.2, 430.6, 431...","[0.11295246688422361, 0.08273628073335591, 0.0...","[29.044920055943216, 37.23132633001016, 37.609...","[90, 91, 94, 93, 92, 120, 134, 138, 143, 147, ...",40.340000,Elton
20,590102440,58536,male,bike,1464.00,"[12.352906242012978, 12.352890148758888, 12.35...","[55.760564832016826, 55.76056374236941, 55.760...","[0.3722291142214761, 10.682818377603066, 31.85...",0,"[-5.2, -5.0, -4.4, -9.2, -11.6, -11.8, -7.6, -...","[0.0010339697617263225, 0.032641945042676035, ...","[0.3722291142214761, 10.682818377603066, 31.85...","[74, 88, 121, 140, 133, 132, 147, 154, 159, 16...",30.650000,George
57,304833487,3714939,male,bike,254.00,"[175.75923115946352, 175.75924817472696, 175.7...","[-40.29023291543126, -40.29022595845163, -40.2...","[2.94831833640659, 6.715151488668759, 11.47518...",2,"[0.0, 0.0, 0.0, 2.2, 4.4, 6.6, 8.8, 8.8, 8.8, ...","[0.0016379546313369945, 0.0018653198579635442,...","[2.94831833640659, 6.715151488668759, 11.47518...","[64, 67, 69, 70, 72, 73, 74, 86, 86, 86, 86, 8...",15.390000,Dennis
89,509658142,4203888,female,bike,643.00,"[12.11823994293809, 12.118284199386835, 12.118...","[55.93290547840297, 55.93292886391282, 55.9330...","[13.646689469555154, 16.87375688953279, 18.527...",2,"[34.0, 34.0, 34.4, 35.4, 36.6, 38.0, 38.6, 39....","[0.0037907470748764316, 0.028122928149221314, ...","[13.646689469555154, 16.87375688953279, 18.527...","[103, 103, 108, 114, 118, 118, 119, 119, 119, ...",33.750000,Wendy
110,421509608,4164701,female,run,306.00,"[-7.6295791659504175, -7.629775805398822, -7.6...","[37.1373672131449, 37.13731097057462, 37.13727...","[17.81356186088025, 10.685093162137749, 10.387...",10,"[29.0, 12.6, 12.8, 12.4, 12.0, 11.6, 12.6, 13....","[0.024741058140111457, 0.01193419598050835, 0....","[17.81356186088025, 10.685093162137749, 10.387...","[70, 78, 86, 93, 97, 100, 107, 110, 118, 123, ...",6.860000,Milly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12534,234772370,9051351,male,bike,722.00,"[-96.0777654312551, -96.07760860584676, -96.07...","[41.234817830845714, 41.234908774495125, 41.23...","[7.508463586673831, 21.8895067147745, 34.25150...",1,"[336.0, 338.0, 335.6, 319.6, 321.2, 324.6, 323...","[0.016685474637052958, 0.05472376678693625, 0....","[7.508463586673831, 21.8895067147745, 39.39670...","[94.0, 107.0, 110.0, 91.0, 86.0, 100.0, 100.0,...",36.771399,Elton
12544,443682294,58536,male,mountain bike,905.00,"[12.432926008477807, 12.432247996330261, 12.43...","[55.764859803020954, 55.764923924580216, 55.76...","[12.913769918091283, 8.61360766179033, 4.35618...",7,"[10.6, 9.2, 11.6, 11.6, 13.8, 15.8, 18.0, 21.8...","[0.04304589972697095, 0.031104694334242854, 0....","[12.913769918091283, 8.61360766179033, 4.35618...","[94.0, 88.0, 104.0, 91.0, 101.0, 115.0, 122.0,...",27.408731,George
12548,567079210,1063624,male,run,617.00,"[21.25957346521318, 21.25957002863288, 21.2594...","[48.971513863652945, 48.97151319310069, 48.971...","[0.9423813987592442, 8.228266387017134, 9.8510...",10,"[286.8, 288.2, 287.0, 286.6, 286.2, 286.2, 284...","[0.013106475330578593, 0.013535761460512481, 0...","[9.436662238016586, 9.85103745992285, 10.01697...","[77.0, 81.0, 84.0, 93.0, 98.0, 102.0, 105.0, 1...",15.903519,Bruce
12549,198268823,1063624,male,run,441.00,"[21.2500491, 21.2500494, 21.2501469, 21.250217...","[48.9931864, 48.993171, 48.9929749, 48.9928267...","[6.1670856897606985, 9.648100228458556, 10.194...",10,"[278.6, 278.6, 277.2, 277.2, 277.2, 277.2, 276...","[0.0017130793582668606, 0.022986227624313176, ...","[6.1670856897606985, 9.648100228458556, 10.194...","[95.0, 95.0, 98.0, 103.0, 106.0, 111.0, 112.0,...",9.

## Get dict function

In [15]:
# Define a function for users who have exercised on more than 1 route
def get_dic(df, Routeid, uid, cityname):
  df1 = df[df['userId'] == uid]
  k = df1[df1['Route_id']==Routeid]
  df_newgen = k.drop_duplicates(subset='distance_adjusted_sum')
  df_newgen = df_newgen.reset_index(drop=True)
  a = []
  for i in range(len(df_newgen)):
    a.append(cityname + str(round(df_newgen['distance_adjusted_sum'][i],2)))
  df_newgen['city'] = a
  dict1 = df_newgen.groupby('id')['city'].apply(list).to_dict()
  return dict1

In [16]:
# Define a function for users who have exercised on only 1 route
def get_singdic(df, uid, cityname):
  df1 = df[df['userId'] == uid]
  df_newgen = df1.drop_duplicates(subset='distance_adjusted_sum')
  df_newgen = df_newgen.reset_index(drop=True)
  a = []
  for i in range(len(df_newgen)):
    a.append(cityname + str(round(df_newgen['distance_adjusted_sum'][i],2)))
  df_newgen['city'] = a
  dict1 = df_newgen.groupby('id')['city'].apply(list).to_dict()
  return dict1

# Run dict


The citynames where 12 users’ routes are located has been found through Google Maps outside. The dictionary will be named through ‘city-country_total route distance ’

In [17]:
dic1 = get_dic(d1, 11, 860533, 'Pretoria-ZA_')
dic2 = get_dic(d1, 10, 860533, 'Cape town-ZA_')
dic3 = get_dic(d1, 8, 1063624, 'Prešov-SVK_')
dic4 = get_dic(d1, 9, 1063624, 'Prešov-SVK_')
dic5 = get_dic(d1, 10, 1063624, 'Prešov-SVK_')
dic6 = get_dic(d1, 11, 1063624, 'Vysoké-SVK_')
dic7 = get_dic(d1, 12, 1063624, 'Prešov-SVK_')
dic8 = get_dic(d1, 13, 1063624, 'Prešov-SVK_')
dic9 = get_dic(d4, 10, 368603, 'Guyancourt-FR_')
dic10 = get_dic(d4, 12, 368603, 'Guyancourt-FR_')
dic11 = get_dic(d4, 9, 368603, 'Versailles-FR_')
dic12 = get_dic(d4, 13, 368603, 'Versailles-FR_')
dic13 = get_dic(d4, 10, 7178673, 'Varde-DK_')
dic14 = get_dic(d4, 12, 7178673, 'Varde-DK_')

In [18]:
run_dic = {**dic1, **dic2, **dic3, **dic4, **dic5, **dic6, **dic7, **dic8, **dic9, **dic10, **dic11, **dic12, **dic13, **dic14 }

In [19]:
print(len(run_dic))

1626


# Bike dict

In [20]:
dic2_1 = get_singdic(d2, 3714939, 'PNTH-NZL_')
dic2_2 = get_dic(d2, 2, 9051351, 'New Jersey-USA_')
dic2_3 = get_dic(d2, 1, 9051351, 'Nebraska-USA_')
dic2_4 = get_singdic(d5, 240120, 'Dallas-USA_')
dic2_5 = get_singdic(d5, 4164701, 'Tavira-Portugal_')

In [21]:
Bike_dic = {**dic2_1, **dic2_2, **dic2_3, **dic2_4, **dic2_5}

In [22]:
print(len(Bike_dic))

1363


# Mountain bike dict

In [23]:
dic3_1 = get_dic(d3, 5, 171814, 'Gjøvik-NO_')
dic3_2 = get_dic(d3, 7, 171814, 'Gjøvik-NO_')
dic3_3 = get_dic(d3, 5, 58536, 'Hørsholm-DK_')
dic3_4 = get_dic(d3, 7, 58536, 'Hørsholm-DK_')
dic3_5 = get_singdic(d6, 6892365, 'Krubin-PL_')
dic3_6 = get_singdic(d6, 4203888, 'Hillerød-DK_')

In [24]:
Mountainbike_dic = {**dic3_1, **dic3_2, **dic3_3, **dic3_4, **dic3_5, **dic3_6}

In [25]:
print(len(Mountainbike_dic))

448


# Total dictionary

In [27]:
dic_whole = {**run_dic, **Bike_dic, **Mountainbike_dic}

In [28]:
print(len(dic_whole))

3437


In [ ]:
dic_whole

# New_df

In [30]:
df['city'] = df['id'].map(dic_whole)

In [31]:
df2 = df.dropna(subset=['city'])

In [32]:
df2

,id,userId,gender,sport,calories,longitude,latitude,derived_speed,Route_id,altitude,derived_distance,speed,heart_rate,distance_adjusted_sum,name,city
8,236905945,9051351,male,bike,1762.83,"[-96.07764498330653, -96.07759008184075, -96.0...","[41.23508454300463, 41.236099088564515, 41.236...","[29.044920055943216, 37.23132633001016, 37.609...",1,"[442.2, 440.6, 437.2, 433.6, 431.2, 430.6, 431...","[0.11295246688422361, 0.08273628073335591, 0.0...","[29.044920055943216, 37.23132633001016, 37.609...","[90, 91, 94, 93, 92, 120, 134, 138, 143, 147, ...",40.340000,Elton,[Nebraska-USA_40.34]
57,304833487,3714939,male,bike,254.00,"[175.75923115946352, 175.75924817472696, 175.7...","[-40.29023291543126, -40.29022595845163, -40.2...","[2.94831833640659, 6.715151488668759, 11.47518...",2,"[0.0, 0.0, 0.0, 2.2, 4.4, 6.6, 8.8, 8.8, 8.8, ...","[0.0016379546313369945, 0.0018653198579635442,...","[2.94831833640659, 6.715151488668759, 11.47518...","[64, 67, 69, 70, 72, 73, 74, 86, 86, 86, 86, 8...",15.390000,Dennis,[PNTH-NZL_15.39]
111,580993447,860533,male,run,539.00,"[28.221429102122784, 28.221432454884052, 28.22...","[-25.723475692793727, -25.723535288125277, -25...","[7.979195494298872, 8.980701845299484, 8.88070...",11,"[1279.4, 1279.8, 1281.0, 1281.0, 1281.0, 1281....","[0.006649329578582394, 0.006593690028639869, 0...","[7.979195494298872, 8.980701845299484, 8.88070...","[94, 97, 101, 104, 108, 111, 115, 119, 120, 12...",8.470000,Christian,[Pretoria-ZA_8.47]
117,196324896,171814,male,mountain bike,1118.34,"[10.6861426, 10.6854682, 10.6844066, 10.684098...","[60.7815276, 60.7819921, 60.7827599, 60.782720...","[13.445568513689267, 16.126901499015162, 10.37...",5,"[162.6, 158.0, 158.0, 158.0, 161.0, 163.0, 165...","[0.06349296242575488, 0.10303298179926354, 0.0...","[13.445568513689267, 16.126901499015162, 10.37...","[133, 134, 124, 115, 122, 130, 137, 145, 149, ...",23.950000,Franklin,[Gjøvik-NO_23.95]
120,86613071,3714939,male,bike,302.00,"[175.75939754024148, 175.75945923104882, 175.7...","[-40.29022025875747, -40.290232663974166, -40....","[9.74322452325506, 13.018138872650102, 14.8908...",2,"[0.0, 0.0, 0.0, -0.4, -0.4, -0.4, -0.4, -1.0, ...","[0.005412902512919478, 0.007232299373694501, 0...","[9.74322452325506, 13.018138872650102, 14.8908...","[80, 83, 85, 86, 89, 88, 88, 88, 88, 88, 89, 9...",15.070000,Dennis,[PNTH-NZL_15.07]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12534,234772370,9051351,male,bike,722.00,"[-96.0777654312551, -96.07760860584676, -96.07...","[41.234817830845714, 41.234908774495125, 41.23...","[7.508463586673831, 21.8895067147745, 34.25150...",1,"[336.0, 338.0, 335.6, 319.6, 321.2, 324.6, 323...","[0.016685474637052958, 0.05472376678693625, 0....","[7.508463586673831, 21.8895067147745, 39.39670...","[94.0, 107.0, 110.0, 91.0, 86.0, 100.0, 100.0,...",36.771399,Elton,[Nebraska-USA_36.77]
12544,443682294,58536,male,mountain bike,905.00,"[12.432926008477807, 12.432247996330261, 12.43...","[55.764859803020954, 55.764923924580216, 55.76...","[12.913769918091283, 8.61360766179033, 4.35618...",7,"[10.6, 9.2, 11.6, 11.6, 13.8, 15.8, 18.0, 21.8...","[0.04304589972697095, 0.031104694334242854, 0....","[12.913769918091283, 8.61360766179033, 4.35618...","[94.0, 88.0, 104.0, 91.0, 101.0, 115.0, 122.0,...",27.408731,George,[Hørsholm-DK_27.41]
12548,567079210,1063624,male,run,617.00,"[21.25957346521318, 21.25957002863288, 21.2594...","[48.971513863652945, 48.97151319310069, 48.971...","[0.9423813987592442, 8.228266387017134, 9.8510...",10,"[286.8, 288.2, 287.0, 286.6, 286.2, 286.2, 284...","[0.013106475330578593, 0.013535761460512481, 0...","[9.436662238016586, 9.85103745992285, 10.01697...","[77.0, 81.0, 84.0, 93.0, 98.0, 102.0, 105.0, 1...",15.903519,Bruce,[Prešov-SVK_15.9]
12549,198268823,1063624,male,run,441.00,"[21.2500491, 21.2500494, 21.2501469, 21.250217...","[48.9931864, 48.993171, 48.9929749, 48.9928267...","[6.1670856897606985, 9.648100228458556, 10.194...",10,"[278.6, 278.6, 277.2, 277.2, 277.2, 

In [34]:
# Save the UI dataset
df2.to_csv('UI_test.csv', index = False)